In [1]:
#%%
import numpy as np
import pandas as pd
import re
from datetime import datetime
import os
import re

np.set_printoptions(threshold=100000)
pd.options.display.max_rows = 500
pd.options.display.max_columns = 100
pd.set_option('expand_frame_repr', False)


# dfx = pd.DataFrame()
# files_dir = os.walk('D:/workspace/zongdi_all')
# r_list = []
# for root, dirs, files in files_dir:
#     for f in files:
#         f1 = re.findall(re.compile('all-.*?xlsx?'), f)
#         f2 = f1[0] if f1 else None
#         if f2:
#             f3 = os.path.join(root, f2)
#             r_list.append(f3)
# print(r_list)

# for r in r_list:
#     df = pd.read_excel(r, sheet_name='Sheet1')
#     dfx = pd.concat([dfx, df])
    
# dfx.drop_duplicates(inplace=True, keep='first',
#     subset=['合同号', '合同字号', '宗地号', '签订日期', '土地用途', '用地面积', '建筑面积说明', '容积率'])
# dfx.reset_index(drop=True, inplace=True)
# dfx.to_excel('D:/workspace/zongdi_all/宗地合并总.xlsx')


In [2]:
df = pd.read_excel('D:/workspace/zongdi_all/宗地合并总.xlsx')
def get_date(contract, col, types):
    contract = str(contract).replace(' ', '')
    date_txt = re.findall(re.compile('(\d{4}年\d{1,2}月\d{1,2}日).?前' + types), contract)
    date_txt2 = re.findall(re.compile('types' + '期?限?延.?至 (\d{4}年\d{1,2}月\d{1,2}日)' ), contract)
    date_txt3 = re.findall(re.compile('types' + '期限调整为 (\d{4}年\d{1,2}月\d{1,2}日)' ), contract)
    date_list = date_txt + date_txt2 + date_txt3
    date_list = [i.replace('年', '-').replace('月', '-').replace('日', '') for i in date_txt]
    z = date_list[-1] if len(date_list)>=2 else col
    z = z.replace('-1-', '-01-').replace('-2-', '-02-').replace('-3-', '-03-').replace('-4-', '-04-').replace('-5-', '-05-')
    z = z.replace('-6-', '-06-').replace('-7-', '-07-').replace('-8-', '-08-').replace('-9-', '-09-')
    return z

def get_date3(contract, col, types):
    contract = str(contract).replace(' ', '')
    date_txt = re.findall(re.compile('(\d{4}年\d{1,2}月\d{1,2}日).?前' + types), contract)
    date_txt2 = re.findall(re.compile('types' + '延.?至 (\d{4}年\d{1,2}月\d{1,2}日)' ), contract)
    date_list = date_txt + date_txt2
    date_list = [i.replace('年', '-').replace('月', '-').replace('日', '') for i in date_txt]
      
    return date_list

def get_date2(contract):
    contract = str(contract).replace(' ', '')
    date_txt = re.findall(re.compile('(\d{4}年\d{1,2}月\d{1,2}日)'), contract)
    date_list = [i.replace('年', '-').replace('月', '-').replace('日', '') for i in date_txt]
    return date_list

def get_finished(contract):
    contract = str(contract).replace(' ', '')
    z = ''
    if '项目已建成' in contract:
        z = '已建成'
    if '现状已建成' in contract:
        z = '已建成'
    if '宗地已建成' in contract:
        z = '已建成'
    if '项目已竣工' in contract:
        z = '已建成'
    if '实际竣工' in contract:
        z = '已建成'
    if '实际建成' in contract: 
        z = '已建成'
    if '视为已竣工' in contract:
        z = '已建成'
    if '《建设工程规划验收合格证》' in contract:
        z = '已建成'
    return z

def compare(con_date, jun_date, jun_date_fix):
    con_date = str(con_date)
    jun_date = str(jun_date)
    jun_date_fix = str(jun_date_fix)
    error = ''
    if (jun_date_fix < con_date) & (jun_date != jun_date_fix):
        error = 'error'
    return error

df['协议内容'] = df['协议内容'].astype(str)
df['开工日期'] = df['开工日期'].astype(str)
df['竣工日期'] = df['竣工日期'].astype(str)
df['开工日期_fix'] = df.apply(lambda x: get_date(x['协议内容'], x['开工日期'], '开工'), axis=1)
df['竣工日期_fix'] = df.apply(lambda x: get_date(x['协议内容'], x['竣工日期'], '竣工'), axis=1)
df['error'] = df.apply(lambda x: compare(x['签订日期'], x['竣工日期'], x['竣工日期_fix']), axis=1)
df['temp_开工'] = df.apply(lambda x: get_date3(x['协议内容'], x['开工日期'], '开工'), axis=1)
df['temp_竣工'] = df.apply(lambda x: get_date3(x['协议内容'], x['竣工日期'], '竣工'), axis=1)
df['temp_日期'] = df['协议内容'].map(lambda x: get_date2(x))
df['是否建成'] = df['协议内容'].map(lambda x: get_finished(x))

# df = df.applymap(lambda x: '' if x=='nan' else x)

df


,合同号,合同字号,供应方式,宗地代码,宗地号,出让方名称,土地用途,签订日期,用地面积,土地性质,土地用途.1,使用年限,开工日期,竣工日期,用地项目名称,无用地方案原因,建筑面积说明,建筑覆盖率,容积率,建筑层数,建筑面积,退红线要求,主体建筑物性质,协议内容,出让金,开发费,配置费,总地价,地价方案备注,受让方名称,受让方地址,减免出让金,减免开发费,减免配置费,减免其他收入,开工日期_fix,竣工日期_fix,error,temp_开工,temp_竣工,temp_日期,是否建成
0,2014-78008,补2,协议,440306201009GB00090,A626-0174,深圳市规划和国土资源委员会,"商业用地,二类居住用地",2019-02-14,36869.8,NaN,"商业用地,二类居住用地",40,nan,nan,宏发天汇城一期,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,深圳市土地使用权出让补充合同书\n深地合字（2014）78008号之补充合同二\n\n一、本...,56501,0,320175,376676,NaN,深圳市天荣盛房地产开发有限公司,深圳市宝安中心区N5区宏发领域花园4栋2717,NaN,NaN,NaN,NaN,nan,nan,,[],[],[],
1,2007-4137,补2,协议,440306205003GB00055,A646-0077,深圳市规划和国土资源委员会光明管理局,工业用地,2019-01-31,52301.7,NaN,工业用地,50,nan,2019-05-08,日东光学厂区,NaN,厂房建筑面积115742㎡，宿舍及其他配套建筑面积15012㎡。,≤45%,≤2.5,40米,130754,1.建筑退用地红线距离大于等于9米。2.绿化覆盖率≥30%。3.按照《深标》要求提供不少于用...,NaN,深 圳 市\n\n土地使用权出让补充合同书\n\n深地合字（2007）4137号...,0,0,0,388802,因竣工期调整，乙方同意缴纳土地违约金（人民币）叁拾捌万捌仟捌佰零贰元整（小写：￥388802...,深圳市日东光学有限公司,深圳市光明区凤凰街道光明高新技术产业园区光源五路1号,NaN,NaN,NaN,NaN,nan,2019-05-08,,[],[2019-5-8],"[2019-5-8, 2019-1-31]",
2,2016-7011,补1,协议,440306201009GB00100,A630-0066,深圳市规划和国土资源委员会光明管理局,环境卫生设施用地,2018-12-10,640.04,NaN,环境卫生设施用地,50,nan,2019-07-27,中粮云景花园环卫站,因开竣工期限调整签订本补充合同，无重新核发用地方案图,总建筑面积310平方米，其中垃圾收集点建筑面积90平方米、公共厕所建筑面积100平方米、再生...,≤30%,≤.48,12米,310,1.建筑退红线要求：北侧≥4米，西侧≥6米2.绿化覆盖率≥60%3.设计方案须符合国家、省、...,NaN,深 圳 市\n\n土地使用权出让补充合同书\n\n深地合字（2016）7011号...,0,0,0,0,该宗地根据深规土规[2018]11号文相关规定调整开竣工期限，无需缴纳违约金,深圳市光明区政府,深圳市光明区光明街道办事处广场路1号,NaN,NaN,NaN,NaN,nan,2019-07-27,,[],"[2018-7-27, 2019-7-27]","[2018-7-27, 2019-7-27, 2018-12-10]",
3,2019-7005,补1,协议,440306201005GB00332,A638-0452,深圳市规划和国土资源委员会光明管理局,二类居住用地,2019-02-01,15612,NaN,二类居住用地,70,2020-01-23,2023-01-23,深圳市公明上村股份合作公司征地返还项目,NaN,规定建筑面积共69650平方米，其中，住宅61900平方米，商业3500平方米，公共配套设施...,≤0%,≤4.46,≤100米,69650,1.建筑退红线要求：一级退线：各侧≥6米，二级退线：各侧≥9米。2.绿化覆盖率≥40%。3....,NaN,深 圳 市\n土地使用权出让补充合同书\n\n深地合字（2019）7005号之补...,2661287,0,15080624,17741911,NaN,深圳市公明上村股份合作公司,深圳市光明新区公明街道上村社区上村村委办公大楼,NaN,NaN,NaN,NaN,2020-01-23,2023-01-23,,[],[],[2019-2-1],
4,2008-8019,补1,挂牌,440306205001GB00488,A603-0358,深圳市规划和国土资源委员会光明管理局,工业用地,2019-01-28,104305,NaN,工业用地,50,nan,2019-10-31,迈瑞光明生产基地,因开竣工期限调整签订本补充合同，无重新核发用地方案图,工业厂房、研发及附属配套设施建筑面积共260764平方米，其中，宿舍及食堂的建筑面积不超过总...,≤40%,≤2.5,≤60米,260764,1、该用地项目建筑方案须进行招标，总体布局以规划主管部门批准的方案为准。2、建筑退红线要求：...,NaN,深 圳 市\n\n土地使用权出让补充合同书\n\n深地合字（2008）8019号...,0,0,0,0,该宗地根据深规土规[2018]11号文相关规定调整开竣工期限，无需缴纳违约金,深圳迈瑞生物医疗电子股份有限公司,深圳市南山区高新技术产业园区科技南十二路迈瑞大厦1-4层,NaN,NaN,NaN,NaN,nan,2019-10-31,,[],"[2015-9-30, 2019-10-31]","[2015-9-30, 2019-10-31, 2019-1-30]",
5,2019-7008,协,协议,440306206007GB00656,A501-0080,深圳市规划和国土资源委员会光明管理局,普通工业用地,2019-01-25,788341,NaN,普通工业用地,30,nan,nan,深圳市华星光电半导体显示技术有限公司A501-0074、A501-0079宗地合宗项目,NaN,其中宿舍≤36000平方米，办公用房≤134000平方米，食堂≤17000平方米，社区警务室...,≤0%,≤4,≤100米,3153368,1.建筑退红线要求：各侧建筑退红线24米以下部分≥6米，24米以上部分≥9米，同时须满足轨道...,NaN,nan,0,0,0,0,本宗地为合宗而来，原A501-0074、A501-0079宗地对应的地价款转为本地块地价款，...,深圳市华星光电半导体显示技术有限公司,深圳市光明区公明街道塘明大道9-2号,NaN,NaN,NaN,NaN,nan,nan,,[],[],[],
6,2019-7007,合,协议,440306201011GB00751,A616-0187,深圳市规划和国土资源委员会光明管理局,新型产业用地,2019-01-31,6004.66,NaN,新型产业用地,50,2020-01-30,2022-01-30,上石家2006-60M-704调入地块,NaN,规定建筑面积25200平方米，其中研发用房18830平方米、公交首末站1800平方米、商业2...,≤50%,≤4.2,≤80米,25200,1.建筑退红线要求：各侧≥9米。2.绿化覆盖率≥30%。3. 按照《光明新区海绵城市专项规划...,研发用房,nan,0,0,0,0,根据《深圳市原农村集体经济组织非农建设用地和征地返还用地土地使用权交易若干规定》（深府〔20...,深圳市马田上石家股份合作公司,深圳市光明区马田街道石家社区上石家股份合作公司办公楼（办公场所）,NaN,NaN,NaN,NaN,2020-01-30,2022-01-30,,[],[],[],
7,2019-7006,合,协议,440306201011GB00750,A616-0186,深圳市规划和国土资源委员会光明管理局,新型产业用地,2019-01-31,6097.08,NaN,新型产业用地,50,2020-01-30,2022-01-30,上石家2006-60M-704调入地块,NaN,规定建筑面积24388平方米，其中，研发用房17088㎡，宿舍6300㎡，文化活动室1000㎡。,≤50%,≤4,≤80米,24388,1.建筑退红线要求：各侧≥9米。2.绿化覆盖率≥30%。3. 按照《光明新区海绵城市专项规划...,研发用房,nan,0,0,0,0,根据《深圳市原农村集体经济组织非农建设用地和征地返还用地土地使用权交易若干规定》（深府〔20...,深圳市马田上石家股份合作公司,深圳市光明区马田街道石家社区上石家股份合作公司办公楼（办公场所）,NaN,NaN,NaN,NaN,2020-01-30,2022-01-30,,[],[],[],
8,2019-7001,划拨,NaN,NaN,A607-0857,光明区人民政府,教育设施用地,2019-01-24,32402.8,NaN,NaN,NaN,2020-01-24,2022-01-24,NaN,NaN,NaN,NaN,1.66,NaN,1.规定建筑面积53639.79平方米，其中教学及教学辅助用房42523.2平方米（包括微格...,1.建筑退红线要求：各侧建筑24米及以下部分退线≥6米，24米以上部分≥9米；2.绿化覆盖率...,Na

In [3]:
df.to_excel('D:/workspace/zongdi_all/宗地合并-提取竣工日期4.xlsx')

In [4]:
df_h = pd.read_excel('D:\工作\开竣工数据整理\删去2017年12月30后数据.xls')
df_h.loc[(df_h['宗地号']=='A608-0128')&(df_h['合同签订日']=='2007-06-30'), '合同签订日'] = '2007-06-20'
df_h['竣工日期'] = df_h['竣工日期'].astype(str)
df_hh = df_h[df_h['竣工日期'].str.len()<5]
for index, rows in df_hh.iterrows():
    zongdi_no = rows['宗地号']
    con = rows['合同签订日']
    dfi = df[(df['宗地号']==zongdi_no)&(df['签订日期']==con)]
    dfi.reset_index(drop=True, inplace=True)
    df_h.at[index, '竣工日期_增' ] = dfi.at[0, '竣工日期_fix']
    df_h.at[index, '是否建成' ] = dfi.at[0, '是否建成']
    
df_h['开工日期'] = df_h['开工日期'].astype(str)
df_hk = df_h[df_h['开工日期'].str.len()<5]
for index, rows in df_hk.iterrows():
    zongdi_no = rows['宗地号']
    con = rows['合同签订日']
    dfi = df[(df['宗地号']==zongdi_no)&(df['签订日期']==con)]
    dfi.reset_index(drop=True, inplace=True)
    df_h.at[index, '开工日期_增' ] = dfi.at[0, '开工日期_fix']

    
df_h.to_excel('D:\工作\开竣工数据整理\删去2017年12月30后数据-3.xls')   
df_h
   


,id,用地单位,宗地号,合同号,合同字号,合同签订日,合同总地价,出让金,开发费,配套费,用地面积,供应方式,协议出让原,土地类别,土地使用年,项目名称,总建筑面积,开工日期,竣工日期,局别,宗地号_1,竣工日期_增,是否建成,开工日期_增
0,322,深圳航空有限责任公司,A001-0126,2007-4195,合,2008-05-23,4.773864e+08,0.000000e+00,0.0,0.000000e+00,92808.28,挂牌,NaN,居住用地,70年0月,NaN,0.00,nan,2012-05-23,宝安管理局,NaN,NaN,NaN,nan
1,323,深圳市航空有限责任公司,A001-0126,深地合字2007-4195,补1,2013-08-26,0.000000e+00,0.000000e+00,0.0,0.000000e+00,92808.28,挂牌,NaN,居住用地,70年0月,NaN,0.00,nan,2014-08-26,宝安管理局,NaN,NaN,NaN,nan
2,324,深圳航空有限责任公司,A001-0126,2007-4195,补2,2014-11-07,0.000000e+00,0.000000e+00,0.0,0.000000e+00,92808.28,NaN,NaN,居住用地,70年0月,NaN,0.00,nan,2015-11-07,宝安管理局,NaN,NaN,NaN,nan
3,325,前海人寿保险股份有限公司,A001-0198,2014-1009,合,2014-04-10,4.000000e+08,6.000000e+07,1303296.0,3.386967e+08,10860.80,挂牌,NaN,社会福利用地,50年0月,NaN,27000.00,nan,2016-04-09,宝安管理局,NaN,NaN,NaN,nan
4,326,"前海人寿保险股份有限公司,深圳市前海幸福之家投资管理有限公司",A001-0198,2014-1009,补1,2014-06-17,0.000000e+00,0.000000e+00,0.0,0.000000e+00,10860.80,挂牌,NaN,社会福利用地,50年0月,NaN,0.00,nan,nan,宝安管理局,NaN,nan,,nan
5,327,深圳市前海幸福之家投资管理有限公司,A001-0198,2014-1009,补2,2017-10-10,0.000000e+00,0.000000e+00,0.0,0.000000e+00,10860.80,挂牌,NaN,社会福利用地,50年0月,幸福之家养老院,0.00,nan,2018-04-08,宝安管理局,NaN,NaN,NaN,nan
6,328,深圳市宝安区人民政府,A002-0003,2009-4061,补1,2009-09-15,0.000000e+00,0.000000e+00,0.0,0.000000e+00,105390.85,协议,NaN,广场用地,50年0月,NaN,0.00,nan,2013-05-25,宝安管理局,NaN,NaN,NaN,nan
7,329,深圳市宝安区人民政府,A002-0003,2009-4061,合,2009-09-15,0.000000e+00,0.000000e+00,0.0,0.000000e+00,105390.85,协议,NaN,广场用地,50年0月,NaN,3940.00,nan,2011-09-15,宝安管理局,NaN,NaN,NaN,nan
8,330,深圳市宝安区人民政府,A002-0003,2009-4061,补2,2012-08-03,0.000000e+00,0.000000e+00,0.0,0.000000e+00,105390.85,协议,NaN,广场用地,0年0月,NaN,0.00,nan,nan,宝安管理局,NaN,nan,,nan
9,331,深圳市华侨城酒店集团有限公司,A002-0030,2007-4026,合,2007-09-14,9.311670e+07,1.396750e+07,2117467.0,7.703173e+07,21174.67,挂牌,NaN,旅馆业用地,50年0月,NaN,0.00,nan,2010-09-14,宝安管理局,NaN,NaN,NaN,nan
